In [197]:
def oracleTopd(query):    
    import oracledb as od
    import pandas as pd

    # db connection
    od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
    conn = od.connect(user = 'admin', password = 'INISW2inisw2',
                      dsn = 'inisw2_high')
    exe = conn.cursor()
    exe.execute(query)
    
    row = exe.fetchall() # row 불러오기

    column_name = exe.description # column 불러오기
    columns=[]
    
    for i in column_name:
        columns.append(i[0])
    
    # row, column을 pandas DataFrame으로 나타내기
    result = pd.DataFrame(row, columns=columns)
    result.rename(mapper=str.lower, axis='columns', inplace=True)
    
    # dtype clob을 string으로 변환
    for clob in result.columns:
        if result[clob].dtype == object:
            result[clob] = result[clob].astype("string")
    
    conn.close()
    
    return result

query = 'select * from recipe_nutrient'
df = oracleTopd(query)
df

,recipe_title,총합_에너지,총합_수분,총합_단백질,총합_지방,총합_회분,총합_탄수화물,총합_당류,총합_식이섬유,총합_칼슘,...,총합_베타카로틴,총합_티아민,총합_리보플라빈,총합_니아신,총합_비타민c,총합_비타민d,총합_콜레스테롤,총합_포화지방산,총합_트랜스지방산,총합_폐기율
0,참치 김치찌개,338.4,184.102,30.828,8.184,5.102,34.224,1.26,0.0,7.325,...,0.0,0.078,0.006,0.168,2.28,0.0,0.0,0.012,0.0,7.2
1,청국장찌개,6910.0,35.0,91.7,719.7,14.9,138.6,39.7,96.0,700.0,...,290.0,6.6,1.3,11.7,11.0,0.0,0.0,61.8,0.0,470.0
2,김치나베돈가스 - 밥 반찬으로 좋아요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,에그베이컨감자전,223.95,13.23,2.935,1.273,4.02,53.55,44.444,5.715,39.3,...,1104.45,0.086,0.34,1.395,0.0,0.0,0.0,0.222,0.008,0.0
4,말복 _ 돼지목살 콩나물불고기 푸짐푸짐하게,57.0,0.255,0.018,0.003,0.03,14.696,13.927,0.0,7.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,동지팥죽 끓이는 법/팥칼국수/red bean porridge/팥 통째로 사용/새알심...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,TV요리) 된장라면 구수~~~~~허다!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,짜장덮밥 짜장밥 만드는법 한그릇 냉파요리,253.55,267.63,4.12,15.168,2.96,25.121,1.93,12.9,109.45,...,1.65,0.05,0.224,0.35,20.4,0.0,0.0,2.258,0.044,0.0
931,뚝배기밥 면역력 쑥쑥~!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [198]:
nutrients_df = df.drop(columns=['recipe_title'])
matrix = nutrients_df.to_numpy()
matrix

array([['338.4', '184.102', '30.828', ..., '0.012', '0.0', '7.2'],
       ['6910.0', '35.0', '91.7', ..., '61.8', '0.0', '470.0'],
       ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ...,
       ['253.55', '267.63', '4.12', ..., '2.258', '0.044', '0.0'],
       ['0.0', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['65.0', '87.4', '3.08', ..., '2.17', '0.14', '0.0']], dtype=object)

In [199]:
matrix.shape

(933, 25)

In [200]:
matrix1 = matrix.copy()

In [201]:
matrix1 = matrix1.astype(np.float32)
matrix1 = torch.tensor(matrix1)

# 결과 확인
print(matrix1)

tensor([[3.3840e+02, 1.8410e+02, 3.0828e+01,  ..., 1.2000e-02, 0.0000e+00,
         7.2000e+00],
        [6.9100e+03, 3.5000e+01, 9.1700e+01,  ..., 6.1800e+01, 0.0000e+00,
         4.7000e+02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [2.5355e+02, 2.6763e+02, 4.1200e+00,  ..., 2.2580e+00, 4.4000e-02,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [6.5000e+01, 8.7400e+01, 3.0800e+00,  ..., 2.1700e+00, 1.4000e-01,
         0.0000e+00]])


In [202]:
matrix1.shape

torch.Size([933, 25])

In [203]:
matrix2 = matrix1[:30]

In [204]:
matrix2.shape

torch.Size([30, 25])

In [205]:
# 차원 추가하여 shape 변경
matrix2 = torch.unsqueeze(matrix2, dim=0)

# 결과 확인
matrix2.shape

torch.Size([1, 30, 25])

In [206]:
# target = torch.randn(output_size)
# target.shape

torch.Size([100])

In [208]:
# 원-핫 벡터 생성
one_hot_vector = torch.eye(30, 100)

one_hot_vector.shape

torch.Size([30, 100])

In [209]:
one_hot_vector[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [210]:
import torch
import torch.nn as nn
import torch.optim as optim

# 하이퍼파라미터 정의
input_size = 25
hidden_size = 50
output_size = 100
# seq_length = 3
# batch_size = 2
learning_rate = 0.001
num_epochs = 10

# 모델 생성
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # 마지막 시퀀스 출력에 대해 선형 레이어 적용
        return out

model = SimpleRNN(input_size, hidden_size, output_size)

# 손실 함수 및 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 더미 입력 데이터와 목표값 생성
# input_data = torch.randn(batch_size, seq_length, input_size)
# target = torch.randn(output_size)

# 모델 학습
for epoch in range(num_epochs):
    optimizer.zero_grad()  # 그래디언트 초기화

    output = model(matrix2)  # 순방향 전파
    loss = criterion(output, one_hot_vector[0])  # 손실 계산
    loss.backward()  # 역전파 실행
    optimizer.step()  # 옵티마이저로 모델 파라미터 업데이트

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.0754
Epoch [2/10], Loss: 0.0584
Epoch [3/10], Loss: 0.0458
Epoch [4/10], Loss: 0.0364
Epoch [5/10], Loss: 0.0294
Epoch [6/10], Loss: 0.0237
Epoch [7/10], Loss: 0.0192
Epoch [8/10], Loss: 0.0155
Epoch [9/10], Loss: 0.0125
Epoch [10/10], Loss: 0.0101


C:\Users\USER\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([1, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
